In [31]:
import pandas as pd
import numpy  as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [32]:
train_DataFrame = pd.read_csv(filepath_or_buffer="Data/MobilePrice/train.csv", 
                    names=["battery_power", "blue", "clock_speed", "dual_sim", "fc", "four_g", "int_memory", "m_dep", "mobile_wt", "n_cores", "pc","px_height","px_width","ram","sc_h","sc_w","talk_time","three_g","touch_screen","wifi","price_range"], skiprows=1)
test_DataFrame = pd.read_csv(filepath_or_buffer="Data/MobilePrice/test.csv", 
                    names=["id","battery_power", "blue", "clock_speed", "dual_sim", "fc", "four_g", "int_memory", "m_dep", "mobile_wt", "n_cores", "pc","px_height","px_width","ram","sc_h","sc_w","talk_time","three_g","touch_screen","wifi"], skiprows=1)
test_AnswerDataFrame = pd.read_csv(filepath_or_buffer="Data/MobilePrice/gender_submission.csv",
                    names=["id", "price_range"], skiprows=1)

print(train_DataFrame)
print(test_DataFrame)
print(test_AnswerDataFrame)

      battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
0               842     0          2.2         0   1       0           7   
1              1021     1          0.5         1   0       1          53   
2               563     1          0.5         1   2       1          41   
3               615     1          2.5         0   0       0          10   
4              1821     1          1.2         0  13       1          44   
...             ...   ...          ...       ...  ..     ...         ...   
1595           1206     0          3.0         1  10       1          30   
1596            832     1          0.5         1   1       1           5   
1597            848     1          2.8         0   1       0           8   
1598           1851     0          2.9         0   0       0          53   
1599           1166     1          0.5         0   7       0          59   

      m_dep  mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  \
0       

In [33]:
def EncodeData(x: pd.DataFrame) -> list:
    """
    对特征向量进行编码
    将pandas的DataFrame结构转换为list
    @x: pandas.DataFrame
    @return: 特征向量
    """
    input_data = [[x["battery_power"][i], x["blue"][i], x["clock_speed"][i], x["dual_sim"][i], x["fc"][i], x["four_g"][i], x["int_memory"][i], x["m_dep"][i], x["mobile_wt"][i], x["n_cores"][i], x["pc"][i], x["px_height"][i], x["px_width"][i], x["ram"][i], x["sc_h"][i], x["sc_w"][i], x["talk_time"][i], x["three_g"][i], x["touch_screen"][i], x["wifi"][i]] for i in range(len(x))]
    return input_data
    

def EncodeTrainData(x: pd.DataFrame) -> tuple[list, list]:
    """
    对测试集进行编码处理
    将pandas的DataFrame结构转换为list
    @x: pandas.DataFrame
    @return: 特征向量，标签向量
    """
    input_data = EncodeData(x)
    label = [x["price_range"][i] for i in range(len(x))]

    return input_data, label

In [34]:
train_x, train_y = EncodeTrainData(train_DataFrame)
# 假设 train_DataFrame 是已经加载的 DataFrame

# 初始化MinMaxScaler
scaler = MinMaxScaler()

# 对特征向量进行归一化处理
train_x = scaler.fit_transform(train_x)  # 这里 train_x 必须是一个 DataFrame

print(train_x)
print(train_y)

[[0.22778891 0.         0.68       ... 0.         0.         1.        ]
 [0.34736139 1.         0.         ... 1.         1.         0.        ]
 [0.04141617 1.         0.         ... 1.         1.         0.        ]
 ...
 [0.23179693 1.         0.92       ... 1.         1.         1.        ]
 [0.90180361 0.         0.96       ... 1.         0.         0.        ]
 [0.44422178 1.         0.         ... 1.         0.         1.        ]]
[1, 2, 2, 2, 1, 1, 3, 0, 0, 0, 3, 3, 1, 2, 0, 0, 3, 3, 1, 1, 3, 3, 1, 0, 1, 2, 3, 3, 2, 0, 3, 0, 1, 3, 0, 1, 1, 3, 2, 2, 2, 3, 1, 1, 0, 1, 3, 1, 0, 3, 2, 2, 3, 1, 2, 2, 0, 3, 1, 2, 1, 0, 0, 2, 2, 3, 3, 3, 0, 3, 3, 2, 0, 3, 0, 1, 0, 0, 3, 1, 3, 2, 3, 1, 1, 1, 0, 3, 2, 2, 3, 2, 0, 0, 1, 3, 0, 2, 0, 1, 3, 1, 1, 0, 0, 1, 3, 3, 3, 3, 1, 1, 0, 2, 3, 3, 2, 1, 0, 1, 2, 3, 3, 3, 2, 3, 2, 1, 3, 0, 3, 1, 2, 1, 2, 2, 1, 3, 0, 0, 2, 0, 3, 2, 0, 3, 1, 3, 2, 2, 1, 3, 1, 2, 0, 1, 0, 0, 3, 1, 2, 2, 0, 0, 2, 3, 1, 2, 3, 1, 3, 1, 2, 2, 3, 2, 0, 3, 2, 1, 0, 0, 2, 3, 3, 